# Deep Learning Course Project - Gesture Recognition


## Done by Sudharma BG 


### Problem Statement


- Imagine you are working as a data scientist at a home electronics company which manufactures state of the art smart televisions. You want to develop a cool feature in the smart-TV that can recognise five different gestures performed by the user which will help users control the TV without using a remote.

- The gestures are continuously monitored by the webcam mounted on the TV. Each gesture corresponds to a specific command:

##### Gesture Corresponding Action

1. Thumbs Up	Increase the volume.
2. Thumbs Down	Decrease the volume.
3. Left Swipe	'Jump' backwards 10 seconds.
4. Right Swipe	'Jump' forward 10 seconds.
5. Stop	Pause the movie.

Each video is a sequence of 30 frames (or images).

#### Objectives:

- Generator: The generator should be able to take a batch of videos as input without any error. Steps like cropping, resizing and normalization should be performed successfully.

- Model: Develop a model that is able to train without any errors which will be judged on the total number of parameters (as the inference(prediction) time should be less) and the accuracy achieved. As suggested by Snehansu, start training on a small amount of data and then proceed further.

In [1]:
#Importing the required libraries 
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
import imageio

In [2]:
#We set the random seed so that the results don't vary drastically.
np.random.seed(30)
import random as rn
import keras as Keras
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)


In this block, you read the folder names for training and validation. You also set the batch_size here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [4]:
train_doc = np.random.permutation(open('train.csv').readlines())
val_doc = np.random.permutation(open('val.csv').readlines())
batch_size = 51

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with img_idx, y,z and normalization such that you get high accuracy.

In [5]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]#create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(source_path)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,18,100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    
                    
                    # Cropped image of above dimension 
                    # (It will not change orginal image) 
                    
                    #image = image.crop((0, 0, 120, 120))
                    image = image.resize(100, 100)
                    
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] /= 255
                    batch_data[folder,idx,:,:,1] /= 255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] /= 255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if((len(source_path)%batch_size)//2==0):
            batch_size = 2
        else:
            batch_size = 1
        num_batches = len(source_path)%batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,18,100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    
                    
                    # Cropped image of above dimension 
                    # (It will not change orginal image) 
                    
                    #image = image.crop((0, 0, 120, 120))
                    image = image.resize(100, 100)
                    
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] /= 255
                    batch_data[folder,idx,:,:,1] /= 255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] /= 255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

In [6]:
curr_dt_time = datetime.datetime.now()
train_path = 'train'
val_path = 'val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10



Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

## Model

#### 1. CNN + RNN Model

In [7]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
Input_shape = (18, 100, 100, 3)
model = Sequential()
model.add(Conv3D(32, (3,3,3), padding='same',
                 input_shape=Input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv3D(32, (3, 3,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2,2)))
model.add(Dropout(0.5))

model.add(Conv3D(64, (3, 3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv3D(64, (3, 3,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

In [8]:
optimiser = Keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 18, 100, 100, 32)  2624      
_________________________________________________________________
activation (Activation)      (None, 18, 100, 100, 32)  0         
_________________________________________________________________
batch_normalization (BatchNo (None, 18, 100, 100, 32)  128       
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 16, 98, 98, 32)    27680     
_________________________________________________________________
activation_1 (Activation)    (None, 16, 98, 98, 32)    0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 98, 98, 32)    128       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 8, 49, 49, 32)     0

In [9]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [10]:
model_name = 'Gesture_recog' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq=1)

LR = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 10,
  verbose = 0, mode = "auto", min_delta = 1e-04, cooldown = 0,
  min_lr = 0)
callbacks_list = [checkpoint, LR]

In [11]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) - 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [13]:
model.fit_generator(train_generator, steps_per_epoch=3, epochs=num_epochs , 
                     verbose=1,callbacks=callbacks_list, validation_data=val_generator,validation_steps=validation_steps, 
                    class_weight=None, workers=1, 
                    initial_epoch=0,use_multiprocessing=True)

W0614 01:54:33.581856 140702984374080 data_adapter.py:839] Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `tf.data.Dataset`.
W0614 01:54:33.616083 140698242230016 data_utils.py:522] multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.


Epoch 1/10

Epoch 00001: saving model to Gesture_recog_2021-06-1401_53_04.833086/model-00001-1.60355-1.00000.h5
1/3 [=========>....................] - ETA: 0s - loss: 1.6036 - categorical_accuracy: 1.0000
Epoch 00001: saving model to Gesture_recog_2021-06-1401_53_04.833086/model-00001-1.60674-0.50000.h5
2/3 [===================>..........] - ETA: 0s - loss: 1.6067 - categorical_accuracy: 0.5000
Epoch 00001: saving model to Gesture_recog_2021-06-1401_53_04.833086/model-00001-1.60523-0.66667.h5
3/3 [==============================] - ETA: 0s - loss: 1.6052 - categorical_accuracy: 0.6667

W0614 01:54:34.231240 140702984374080 data_adapter.py:839] Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `tf.data.Dataset`.
W0614 01:54:34.282564 140697860552448 data_utils.py:522] multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.


3/3 [==============================] - 1s 199ms/step - loss: 1.6052 - categorical_accuracy: 0.6667 - val_loss: 1.6119 - val_categorical_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 2/10

Epoch 00002: saving model to Gesture_recog_2021-06-1401_53_04.833086/model-00002-1.60486-0.00000.h5
1/3 [=========>....................] - ETA: 0s - loss: 1.6049 - categorical_accuracy: 0.0000e+00
Epoch 00002: saving model to Gesture_recog_2021-06-1401_53_04.833086/model-00002-1.60350-0.33333.h5
2/3 [===================>..........] - ETA: 0s - loss: 1.6035 - categorical_accuracy: 0.3333    
Epoch 00002: saving model to Gesture_recog_2021-06-1401_53_04.833086/model-00002-1.60600-0.20000.h5
3/3 [==============================] - ETA: 0s - loss: 1.6060 - categorical_accuracy: 0.2000

W0614 01:54:35.434858 140702984374080 data_adapter.py:839] Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `tf.data.Dataset`.
W0614 01:54:35.491008 140697751512832 data_utils.py:522] multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.


...

3/3 [==============================] - 2s 617ms/step - loss: 1.6105 - categorical_accuracy: 0.1667 - val_loss: 1.6050 - val_categorical_accuracy: 0.5000 - lr: 0.0010
Epoch 9/10

Epoch 00009: saving model to Gesture_recog_2021-06-1401_53_04.833086/model-00009-1.60754-0.00000.h5
1/3 [=========>....................] - ETA: 0s - loss: 1.6075 - categorical_accuracy: 0.0000e+00
Epoch 00009: saving model to Gesture_recog_2021-06-1401_53_04.833086/model-00009-1.61309-0.00000.h5
2/3 [===================>..........] - ETA: 0s - loss: 1.6131 - categorical_accuracy: 0.0000e+00
Epoch 00009: saving model to Gesture_recog_2021-06-1401_53_04.833086/model-00009-1.61040-0.16667.h5
3/3 [==============================] - ETA: 0s - loss: 1.6104 - categorical_accuracy: 0.1667    

W0614 01:54:47.409531 140702984374080 data_adapter.py:839] Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `tf.data.Dataset`.
W0614 01:54:47.473240 140696736487168 data_utils.py:522] multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.


3/3 [==============================] - 1s 486ms/step - loss: 1.6104 - categorical_accuracy: 0.1667 - val_loss: 1.6091 - val_categorical_accuracy: 0.2500 - lr: 0.0010
Epoch 10/10

Epoch 00010: saving model to Gesture_recog_2021-06-1401_53_04.833086/model-00010-1.60348-0.00000.h5
1/3 [=========>....................] - ETA: 0s - loss: 1.6035 - categorical_accuracy: 0.0000e+00
Epoch 00010: saving model to Gesture_recog_2021-06-1401_53_04.833086/model-00010-1.60418-0.25000.h5
2/3 [===================>..........] - ETA: 0s - loss: 1.6042 - categorical_accuracy: 0.2500    
Epoch 00010: saving model to Gesture_recog_2021-06-1401_53_04.833086/model-00010-1.60570-0.16667.h5
3/3 [==============================] - ETA: 0s - loss: 1.6057 - categorical_accuracy: 0.1667

W0614 01:54:49.011877 140702984374080 data_adapter.py:839] Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `tf.data.Dataset`.
W0614 01:54:49.128597 140696702916352 data_utils.py:522] multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.


3/3 [==============================] - 1s 492ms/step - loss: 1.6057 - categorical_accuracy: 0.1667 - val_loss: 1.6132 - val_categorical_accuracy: 0.0000e+00 - lr: 0.0010


### 2. CNN+LSTM

In [14]:
#write your model here
from keras.models import Sequential, Model
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout, LSTM, Bidirectional


Input_shape_1 = (18, 100, 100, 3)
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (7, 7), strides=(2, 2), activation='relu', padding='same'), input_shape=Input_shape_1))
model.add(TimeDistributed(Conv2D(32, (3,3), kernel_initializer="he_normal", activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
 
model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
 
model.add(TimeDistributed(Conv2D(128, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(128, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
 
model.add(TimeDistributed(Conv2D(256, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(256, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
 
model.add(TimeDistributed(Conv2D(512, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(512, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
 
model.add(TimeDistributed(Flatten()))
 
model.add(Dropout(0.5))
model.add(LSTM(512, return_sequences=False, dropout=0.5))
model.add(Dense(5, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 18, 50, 50, 32)    4736      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 18, 48, 48, 32)    9248      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 18, 24, 24, 32)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 18, 24, 24, 64)    18496     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 18, 24, 24, 64)    36928     
_________________________________________________________________
time_distributed_5 (TimeDist (None, 18, 12, 12, 64)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 18, 12, 12, 128)  

In [15]:
#optimiser = Keras.optimizers.Adam(lr=0.001)
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 18, 50, 50, 32)    4736      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 18, 48, 48, 32)    9248      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 18, 24, 24, 32)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 18, 24, 24, 64)    18496     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 18, 24, 24, 64)    36928     
_________________________________________________________________
time_distributed_5 (TimeDist (None, 18, 12, 12, 64)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 18, 12, 12, 128)  

In [16]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [17]:
model_name = 'Gesture_recog' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq=1)

LR = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 10,
  verbose = 0, mode = "auto", min_delta = 1e-04, cooldown = 0,
  min_lr = 0)
callbacks_list = [checkpoint, LR]

In [18]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [19]:
model.fit(train_generator, steps_per_epoch=3, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None,initial_epoch=0)

Source path =  train ; batch size = 51
Epoch 1/10

Epoch 00001: saving model to Gesture_recog_2021-06-1401_53_04.833086/model-00001-1.60944-0.00000.h5
1/3 [=========>....................] - ETA: 0s - loss: 1.6094 - categorical_accuracy: 0.0000e+00
Epoch 00001: saving model to Gesture_recog_2021-06-1401_53_04.833086/model-00001-1.61214-0.00000.h5
2/3 [===================>..........] - ETA: 0s - loss: 1.6121 - categorical_accuracy: 0.0000e+00
Epoch 00001: saving model to Gesture_recog_2021-06-1401_53_04.833086/model-00001-1.61525-0.00000.h5
3/3 [==============================] - 2s 525ms/step - loss: 1.6153 - categorical_accuracy: 0.0000e+00 - val_loss: 1.6282 - val_categorical_accuracy: 0.0000e+00 - lr: 0.0100
Epoch 2/10

Epoch 00002: saving model to Gesture_recog_2021-06-1401_53_04.833086/model-00002-1.62902-0.00000.h5
1/3 [=========>....................] - ETA: 0s - loss: 1.6290 - categorical_accuracy: 0.0000e+00
Epoch 00002: saving model to Gesture_recog_2021-06-1401_53_04.833086/mod

W0614 01:55:29.413332 140702984374080 callbacks.py:307] Method (on_train_batch_end) is slow compared to the batch update (0.103332). Check your callbacks.



Epoch 00002: saving model to Gesture_recog_2021-06-1401_53_04.833086/model-00002-1.61309-0.00000.h5
3/3 [==============================] - 2s 514ms/step - loss: 1.6131 - categorical_accuracy: 0.0000e+00 - val_loss: 1.6258 - val_categorical_accuracy: 0.0000e+00 - lr: 0.0100
Epoch 3/10

Epoch 00003: saving model to Gesture_recog_2021-06-1401_53_04.833086/model-00003-1.59274-0.50000.h5
1/3 [=========>....................] - ETA: 0s - loss: 1.5927 - categorical_accuracy: 0.5000
Epoch 00003: saving model to Gesture_recog_2021-06-1401_53_04.833086/model-00003-1.59692-0.50000.h5
2/3 [===================>..........] - ETA: 0s - loss: 1.5969 - categorical_accuracy: 0.5000
Epoch 00003: saving model to Gesture_recog_2021-06-1401_53_04.833086/model-00003-1.59214-0.50000.h5
3/3 [==============================] - 2s 517ms/step - loss: 1.5921 - categorical_accuracy: 0.5000 - val_loss: 1.6116 - val_categorical_accuracy: 0.5000 - lr: 0.0100
Epoch 4/10

Epoch 00004: saving model to Gesture_recog_2021-0